In [1]:
import ssl 

# Disable SSL verification
ssl._create_default_https_context = ssl._create_unverified_context

from Bio import Entrez, SeqIO 

# Fetch the E. coli MG1655 genome
def fetch_ecoli_genome(): 
    '''
    This function is used to obtain the genome data of E. coli MG1655 from the NCBI database. It retrieves data by the specified GenBank ID ("U00096.3")
    '''
    
    Entrez.email = "xxx@qq.com"  
    handle = Entrez.efetch(db="nucleotide", id="U00096.3", rettype="gb", retmode="text") 
    record = SeqIO.read(handle, "genbank")
    handle.close()
    return record

def find_gene_position(record, gene_name):
    for feature in record.features:
        if feature.type == "CDS" and "gene" in feature.qualifiers:
            if feature.qualifiers["gene"][0] == gene_name:
                return feature.location.start, feature.location.end
    return None  

record = fetch_ecoli_genome()


In [2]:
## This module is used to extract specific genes in the genome
genes_to_extract =['gpmA', 'fbaA', 'pykF', 'pfkB', 'pykA', 'gpmM', 'pfkA', 'fbaB']
gene_name = "gpmA"
gene_position = find_gene_position(record, gene_name)

if gene_position is not None:
    gene_seq = record.seq[gene_position[0]-100:gene_position[1]].lower()
    print(f"Found gene {gene_name} at positions {gene_position[0]} to {gene_position[1]}")
    print(f"Gene sequence:\n{gene_seq.upper()}")
    print('Length: ',len(gene_seq.upper()))
    
    print(f"Found gene {gene_name} -100:300 position:")
    gene_truncate = record.seq[gene_position[0]-100:gene_position[0]+300].lower()
    print(gene_truncate.upper())
    print('Length: ',len(gene_truncate.upper()))

else:
    print(f"Gene {gene_name} not found in the genome.") 

Found gene gpmA at positions 786842 to 787595
Gene sequence:
GATAAGTCGCGCAGCGTCGCATCAGGCAATGTGCTCCATTGTTAGCAACAAAAAAGCCGACTCACTTGCAGTCGGCTTTCTCATTTTAAACGAATGACGTTTACTTCGCTTTACCCTGGTTTGCAACCGCCGCTGCTTTCGCTGCGATCTCGTCAGCATTACCCAGATAATAGCGTTTCAGCGGTTTGAAATTCTCGTCGAACTCATACACCAGCGGCACGCCAGTCGGGATATTAAGCTCAAGAATCTCTTCTTCGCTCATGTTATCAAGATATTTCACCAGCGCACGTAAAGAGTTACCGTGTGCAGCGATGATCACGCGCTCACCGCTCTTCATACGCGGCAGAATAGTTTCATTCCAGTAAGGGATCACGCGGTCAATGGTCAGCGCCAGGCTTTCCGTCAGCGGCAGTTCTTTCTCGCTCAGTTTCGCGTAACGCGGATCGTGACCCGGATAACGCTCATCATCTTTAGTCAGTTCCGGCGGAGTCACTGCAAAACCACGACGCCACTGTTTCACCTGCTCGTCGCCATACTTTTCAGCAGTTTCCGCTTTGTTCAGACCCTGCAACGCACCGTAGTGACGTTCGTTCAGTTTCCAGGATTTCTCAACGGGCAGCCATGCCTGATCCAGTTCGTCCAGCACATTCCACAGGGTATGGATAGCGCGTTTCAGCACAGAAGTGTAAGCAAAGTCAAAGCTGTAACCTTCCTCTTTCAGCAGCTTACCTGCTGCTTTTGCTTCGCTTACGCCTTTCTCAGACAGATCCACGTCGTACCAACCGGTGAAACGGTTTTCTTTGTTCCACTGACTTTCGCCATGACGAACCAGAACCAGCTTAGTTACAGCCAT
Length:  853
Found gene gpmA -100:300 position:
GATAAGTCGCGCAGCGTCGCATCAGGCAATGTGCTCC

In [3]:
## This module is used to extract and save gene sequences.
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
def save_as_fasta(seq, gene_name, start, end, filename):
    fasta_record = SeqRecord(Seq(seq.upper()), id=gene_name, description=f"{gene_name} gene from position {start} to {end}")
    with open(filename, "w") as output_handle:
        SeqIO.write(fasta_record, output_handle, "fasta")
    print(f"FASTA file saved as {filename}")

# Save gene sequence as FASTA
save_as_fasta(gene_seq, gene_name, gene_position[0], gene_position[1], f"{gene_name}.fasta")

FASTA file saved as gpmA.fasta


In [4]:
## This module is used to extract and preserve genome sequences
def save_genome_as_fasta(record, filename):
    genome_seq = record.seq
    genome_record = SeqRecord(genome_seq, id=record.id, description="E. coli MG1655 complete genome")
    with open(filename, "w") as output_handle:
        SeqIO.write(genome_record, output_handle, "fasta")
    print(f"FASTA file of the complete genome saved as {filename}")
save_genome_as_fasta(record, "ecoli_mg1655_genome.fasta")

FASTA file of the complete genome saved as ecoli_mg1655_genome.fasta


In [5]:
## This module is used to calculate the average length of extracted genes
genes_to_extract =['gpmA', 'fbaA', 'pykF', 'pfkB', 'pykA', 'gpmM', 'pfkA', 'fbaB']
gene_long = 0
for gene_name in genes_to_extract:
    gene_position = find_gene_position(record, gene_name)
    if gene_position is not None:
        gene_seq = record.seq[gene_position[0]-100:gene_position[1]].lower()
        gene_long += len(gene_seq)
print(gene_long)
print(gene_long/8)

9980
1247.5
